# Web Scrapping

In [ ]:
import requests

url = "https://filkom.ub.ac.id/profile/dosen?p=1"
response = requests.get(url)
print(response.status_code)  # Memastikan statusnya 200
print(response.text[:20000])  # Lihat sebagian isi halaman

200


<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Daftar SDM Dosen</title>

    <link rel="stylesheet" href="https://filkom.ub.ac.id/profile/style-list-responsive.css">
    <link href='https://fonts.googleapis.com/css?family=Lexend' rel='stylesheet'>
    <script src="https://code.iconify.design/iconify-icon/1.0.7/iconify-icon.min.js"></script>

    <!-- Icons? -->
    <link rel="shortcut icon" href="//filkom.ub.ac.id/wp-content/uploads/2021/03/ptiik-1.png" type="image/x-icon">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.2.0/css/all.min.css">


</head>

<body>

    <!-- header -->
    <div class="header">
        <div class="header-top container">
            <a href="">UB Official</a>
            <a href="">Webmail UB</a>
            <a href="">UB News</a>
            <a href="">BITS</a>
        </div>
        <div class="header-main c

In [ ]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Base URL untuk indeks dan profil dosen
base_url = "https://filkom.ub.ac.id/profile/dosen?p="
profile_base_url = "https://filkom.ub.ac.id/profile/dosen/"

# File output untuk menyimpan hasil
output_file = "dosen_data_extended.csv"

# Buka file output untuk menulis hasil
with open(output_file, mode='w', newline='', encoding='utf-8') as output_csv:
    writer = csv.writer(output_csv)
    writer.writerow(['Dosen ID', 'Nama', 'Departemen', 'Prodi', 'Email', 'Publikasi', 'Riset', 'Pengabdian',
                     'Bidang Keilmuwan', 'Riwayat Mengajar'])  # Tambahkan header baru

    page = 1
    while True:
        # URL halaman indeks dosen
        url = f"{base_url}{page}"
        print(f"Mengakses halaman: {url}")
        response = requests.get(url)

        # Periksa apakah halaman dapat diakses
        if response.status_code != 200:
            print(f"Halaman {page} tidak dapat diakses (status: {response.status_code}).")
            break

        # Parsing HTML menggunakan BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Cari elemen ID dosen
        containers = soup.find_all('div', class_='container')
        if not containers:
            print(f"Tidak ada dosen ditemukan di halaman {page}. Menghentikan iterasi.")
            break

        # Proses setiap dosen di halaman
        ids_found = 0
        for container_div in containers:
            rows = container_div.find_all('div', class_='row')
            for row_div in rows:
                dosen_id = row_div.get('id')  # Ambil atribut ID
                if dosen_id:
                    dosen_id_clean = dosen_id.replace("data-", "")
                    profile_url = f"{profile_base_url}{dosen_id_clean}"
                    print(f"Mengakses profil: {profile_url}")
                    profile_response = requests.get(profile_url)

                    # Periksa apakah profil dapat diakses
                    if profile_response.status_code != 200:
                        print(f"Gagal mengakses profil {dosen_id_clean} (status: {profile_response.status_code}).")
                        continue

                    # Parsing profil dosen
                    profile_soup = BeautifulSoup(profile_response.text, 'html.parser')

                    # Ekstraksi Nama
                    nama = profile_soup.find('div', class_='profil').find('h2').text.strip() if profile_soup.find('div', class_='profil') else np.nan

                    # Ekstraksi Departemen dan Prodi
                    departemen = prodi = np.nan
                    data_departemen = profile_soup.find('div', class_='data_departemen')
                    if data_departemen:
                        p_tags = data_departemen.find_all('p')
                        if len(p_tags) > 0:
                            departemen = p_tags[0].text.strip()
                        if len(p_tags) > 1:
                            prodi = p_tags[1].text.strip()

                    # Ekstraksi Email
                    email = np.nan
                    data_kontak = profile_soup.find('div', class_='data_kontak')
                    if data_kontak:
                        email_container = data_kontak.find('div', class_='container')
                        if email_container:
                            email_tag = email_container.find('p')
                            if email_tag:
                                email = email_tag.text.strip()

                    # Ekstraksi Publikasi
                    publikasi_div = profile_soup.find('div', class_='publikasi', id='block-publikasi')
                    publikasi = [h3.text.strip() for h3 in publikasi_div.find_all('h3')] if publikasi_div else []

                    # Ekstraksi Riset
                    riset_div = profile_soup.find('div', class_='riset', id='block-riset')
                    riset = [h3.text.strip() for h3 in riset_div.find_all('h3')] if riset_div else []

                    # Ekstraksi Pengabdian
                    pengabdian_div = profile_soup.find('div', class_='pengabdian', id='block-pengabdian')
                    pengabdian = [h3.text.strip() for h3 in pengabdian_div.find_all('h3')] if pengabdian_div else []

                    # Ekstraksi Bidang Keilmuwan
                    bidang_keilmuwan_div = profile_soup.find('div', class_='row_keilmuan')

                    # Jika div ditemukan, ambil teks di dalam <p> dan pisahkan berdasarkan koma
                    if bidang_keilmuwan_div:
                        bidang_keilmuwan_text = bidang_keilmuwan_div.find('p').text.strip() if bidang_keilmuwan_div.find('p') else ""
                        bidang_keilmuwan = [item.strip() for item in bidang_keilmuwan_text.split(',')] if bidang_keilmuwan_text else []
                    else:
                        bidang_keilmuwan = []

                    # Cek apakah bidang keilmuwan berisi hanya tanda "-"
                    if bidang_keilmuwan == ['-']:
                        bidang_keilmuwan_str = np.nan
                    else:
                        bidang_keilmuwan_str = "; ".join(bidang_keilmuwan) if bidang_keilmuwan else np.nan

                    # Ekstraksi Riwayat Mengajar
                    riwayat_mengajar_div = profile_soup.find('div', class_='row_matkul')
                    riwayat_mengajar = [li.text.strip() for li in riwayat_mengajar_div.find_all('li')] if riwayat_mengajar_div else []

                    # Gabungkan daftar menjadi string dipisahkan koma
                    publikasi_str = "; ".join(publikasi) if publikasi else np.nan
                    riset_str = "; ".join(riset) if riset else np.nan
                    pengabdian_str = "; ".join(pengabdian) if pengabdian else np.nan
                    bidang_keilmuwan_str = "; ".join(bidang_keilmuwan) if bidang_keilmuwan else np.nan
                    riwayat_mengajar_str = "; ".join(riwayat_mengajar) if riwayat_mengajar else np.nan

                    # Tulis hasil ke file CSV
                    writer.writerow([dosen_id_clean, nama, departemen, prodi, email, publikasi_str, riset_str, pengabdian_str,
                                     bidang_keilmuwan_str, riwayat_mengajar_str])
                    ids_found += 1

        # Jika tidak ada ID ditemukan, hentikan iterasi
        if ids_found == 0:
            print(f"Tidak ada dosen ditemukan di halaman {page}. Menghentikan iterasi.")
            break

        print(f"Halaman {page} selesai diproses, ditemukan {ids_found} ID dosen.")
        page += 1

Mengakses halaman: https://filkom.ub.ac.id/profile/dosen?p=1
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/achmad.arwan
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/achmad.basuki
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/achmad.ridok
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/adam.hendra.brata
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/adhitya.bhawiyuga
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/aditya.rachmadi
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/admaja.dwi.herlambang
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/agi.putra.kharisma
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/agung.setia.budi
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/agus.wahyu.widodo
Halaman 1 selesai diproses, ditemukan 10 ID dosen.
Mengakses halaman: https://filkom.ub.ac.id/profile/dosen?p=2
Mengakses profil: https://filkom.ub.ac.id/profile/dosen/alfi.nur.rusydi
Mengakses profil: https://f

In [ ]:
# Baca hasilnya sebagai dataframe
profile_df = pd.read_csv(output_file)

# Ganti "Tidak Ditemukan" dengan NaN di seluruh dataframe
profile_df.replace("Tidak Ditemukan", np.nan, inplace=True)

# Tampilkan dataframe
profile_df

,Dosen ID,Nama,Departemen,Prodi,Email,Publikasi,Riset,Pengabdian,Bidang Keilmuwan,Riwayat Mengajar
0,achmad.arwan,"Dr. Achmad Arwan, S.Kom., M.Kom.",Departemen Teknik Informatika,Program Studi Teknik Informatika,arwan@ub.ac.id,Using Strongly Typed Genetic Programming for k...,2019. Studi Eksplorasi Hubungan Web Accesibili...,2021. Pelatihan Pemodelan Kebutuhan Perangkat ...,Mining Repositori Software; Software Testing,NaN
1,achmad.basuki,"Achmad Basuki, S.T., MMG., Ph.D.",Departemen Teknik Informatika,Program Studi Ilmu Komputer,abazh@ub.ac.id,The Extended Dijkstra's-based Load Balancing f...,2022. Mekanisme Penjaminan Kerahasiaan dan Int...,2021. Pelatihan Pengelolaan Layanan Dasar Clou...,Internet of Things; Software-Defined Networkin...,NaN
2,achmad.ridok,"Dr. Drs. Ir. Achmad Ridok, M.Kom.",Departemen Teknik Informatika,Program Studi Teknik Informatika,acridokb@ub.ac.id,Peringkasan Dokumen Bahasa Indonesia Berbasis ...,2013. Evaluasi Detektor-Deskriptor Untuk Kasus...,2021. Implementasi Sistem Keamanan dengan Clos...,Sistem Temu Kembali Informasi; Bioinformatika;...,NaN
3,adam.hendra.brata,"Ir. Adam Hendra Brata, S.Kom., M.T., M.Sc.",Departemen Teknik Informatika,Program Studi Teknik Informatika,adam@ub.ac.id,An Exploratory Study of Requirements Engineeri...,2022. Implementasi Metode Bates Dalam Purwarup...,2021. Pelatihan Sistem Pengelolaan Nilai Kurik...,-,NaN
4,adhitya.bhawiyuga,"Adhitya Bhawiyuga, S.Kom., M.Sc.",Departemen Teknik Informatika,Program Studi Teknik Informatika,bhawiyuga@ub.ac.id,NaN,2022. Mekanisme Penjaminan Kerahasiaan dan Int...,2021. Pelatihan Pengelolaan Layanan Dasar Clou...,internet of things; cloud computing; cybersecu...,NaN
...,...,...,...,...,...,...,...,...,...,...
103,widhy.hayuhardhika.nugraha.putra,"Ir. Widhy Hayuhardhika Nugraha Putra, S.Kom., ...",Departemen Sistem Informasi,Program Studi Teknologi Informasi,widhy@ub.ac.id,Peringkasan Otomatis Dokumen Teks Berbahasa In...,2022. Pengembangan Aplikasi Mobile Berbasis An...,2021. Pemanfaatan NFT Marketplace Sebagai Alat...,Pengembangan Sistem Informasi; Integrasi Tekno...,NaN
104,wijaya.kurniawan,"Wijaya Kurniawan, S.T., M.T.",Departemen Teknik Informatika,Program Studi Teknik Komputer,wjaykurnia@ub.ac.id,Implementasi Sensor Cahaya Sebagai Pengontrol ...,2019. Implementasi Arsitektur CPU 8-bit Berdas...,2020. Pelatihan Perancangan Algoritma Kendaraa...,Sistem Kendali,NaN
105,yuita.arum.sari,"Yuita Arum Sari, S.Kom., M.Kom.",Departemen Teknik Informatika,Program Studi Teknik Informatika,yuita@ub.ac.id,Human tracking by u sing multiple methods and ...,2019. Analisis Konsistensi Ranking dalam M-hea...,2021. Peningkatan Kemampuan Pengajaran Daring ...,Artificial Intelligence; Computer Vision; Mach...,Induksi Riset; Jaringan Saraf Tiruan; Kalkulus...
106,yusi.tyroni.mursityo,"Ir. Yusi Tyroni Mursityo, S.Kom., M.AB.",Departemen Sistem Informasi,Program Studi Sistem Informasi,yusi_tyro@ub.ac.id,Prediksi Cuaca pada Data Time Series Menggun...,2022. Pengembangan Aplikasi Mobile Berbasis An...,2021. Pelatihan Penggunaan Google Form Untuk P...,Information System Engineering; Business Proce...,NaN


# Preprocessing

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=055a4ba2dbedba03ee73eb974f6d17ac5d038f311495a488e65d63dd9cb55047
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 13.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from langdetect import detect
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

wnl = WordNetLemmatizer()  # Lemmatizer untuk Bahasa Inggris
factory = StemmerFactory()
stemmer_id = factory.create_stemmer()  # Stemmer untuk Bahasa Indonesia

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import numpy as np

# Fungsi untuk menghapus "Tahun. " dan memisahkan berdasarkan ";", hasilnya dalam bentuk list
def preprocess_column_to_list(text):
    if isinstance(text, str):
        # Hapus "Tahun. " di awal setiap elemen
        cleaned_text = text.split(';')
        cleaned_text = [item.strip().lstrip('0123456789').lstrip('. ').strip() for item in cleaned_text]

        # Jika ditemukan "-" dalam list, ganti dengan NaN
        if all(item == "-" for item in cleaned_text):
            return np.nan

        return cleaned_text
    return text

profile_df_preprocessed = profile_df.copy()

# Preprocessing untuk kolom Publikasi, Riset, Pengabdian, Riwayat Mengajar, dan Bidang Keilmuwan
profile_df_preprocessed['Publikasi'] = profile_df_preprocessed['Publikasi'].apply(preprocess_column_to_list)
profile_df_preprocessed['Riset'] = profile_df_preprocessed['Riset'].apply(preprocess_column_to_list)
profile_df_preprocessed['Pengabdian'] = profile_df_preprocessed['Pengabdian'].apply(preprocess_column_to_list)
profile_df_preprocessed['Riwayat Mengajar'] = profile_df_preprocessed['Riwayat Mengajar'].apply(preprocess_column_to_list)
profile_df_preprocessed['Bidang Keilmuwan'] = profile_df_preprocessed['Bidang Keilmuwan'].apply(preprocess_column_to_list)

# Menampilkan dataframe yang sudah diproses
profile_df_preprocessed

,Dosen ID,Nama,Departemen,Prodi,Email,Publikasi,Riset,Pengabdian,Bidang Keilmuwan,Riwayat Mengajar
0,achmad.arwan,"Dr. Achmad Arwan, S.Kom., M.Kom.",Departemen Teknik Informatika,Program Studi Teknik Informatika,arwan@ub.ac.id,[Using Strongly Typed Genetic Programming for ...,[Studi Eksplorasi Hubungan Web Accesibility da...,[Pelatihan Pemodelan Kebutuhan Perangkat Lunak...,"[Mining Repositori Software, Software Testing]",NaN
1,achmad.basuki,"Achmad Basuki, S.T., MMG., Ph.D.",Departemen Teknik Informatika,Program Studi Ilmu Komputer,abazh@ub.ac.id,[The Extended Dijkstra's-based Load Balancing ...,[Mekanisme Penjaminan Kerahasiaan dan Integrit...,[Pelatihan Pengelolaan Layanan Dasar Cloud Com...,"[Internet of Things, Software-Defined Networki...",NaN
2,achmad.ridok,"Dr. Drs. Ir. Achmad Ridok, M.Kom.",Departemen Teknik Informatika,Program Studi Teknik Informatika,acridokb@ub.ac.id,[Peringkasan Dokumen Bahasa Indonesia Berbasis...,[Evaluasi Detektor-Deskriptor Untuk Kasus Obje...,[Implementasi Sistem Keamanan dengan Closed-Ci...,"[Sistem Temu Kembali Informasi, Bioinformatika...",NaN
3,adam.hendra.brata,"Ir. Adam Hendra Brata, S.Kom., M.T., M.Sc.",Departemen Teknik Informatika,Program Studi Teknik Informatika,adam@ub.ac.id,[An Exploratory Study of Requirements Engineer...,[Implementasi Metode Bates Dalam Purwarupa Apl...,[Pelatihan Sistem Pengelolaan Nilai Kurikulum ...,NaN,NaN
4,adhitya.bhawiyuga,"Adhitya Bhawiyuga, S.Kom., M.Sc.",Departemen Teknik Informatika,Program Studi Teknik Informatika,bhawiyuga@ub.ac.id,NaN,[Mekanisme Penjaminan Kerahasiaan dan Integrit...,[Pelatihan Pengelolaan Layanan Dasar Cloud Com...,"[internet of things, cloud computing, cybersec...",NaN
...,...,...,...,...,...,...,...,...,...,...
103,widhy.hayuhardhika.nugraha.putra,"Ir. Widhy Hayuhardhika Nugraha Putra, S.Kom., ...",Departemen Sistem Informasi,Program Studi Teknologi Informasi,widhy@ub.ac.id,[Peringkasan Otomatis Dokumen Teks Berbahasa I...,[Pengembangan Aplikasi Mobile Berbasis Android...,[Pemanfaatan NFT Marketplace Sebagai Alat Penj...,"[Pengembangan Sistem Informasi, Integrasi Tekn...",NaN
104,wijaya.kurniawan,"Wijaya Kurniawan, S.T., M.T.",Departemen Teknik Informatika,Program Studi Teknik Komputer,wjaykurnia@ub.ac.id,[Implementasi Sensor Cahaya Sebagai Pengontrol...,[Implementasi Arsitektur CPU 8-bit Berdasarkan...,[Pelatihan Perancangan Algoritma Kendaraan Oto...,[Sistem Kendali],NaN
105,yuita.arum.sari,"Yuita Arum Sari, S.Kom., M.Kom.",Departemen Teknik Informatika,Program Studi Teknik Informatika,yuita@ub.ac.id,[Human tracking by u sing multiple methods and...,[Analisis Konsistensi Ranking dalam M-health R...,[Peningkatan Kemampuan Pengajaran Daring pada ...,"[Artificial Intelligence, Computer Vision, Mac...","[Induksi Riset, Jaringan Saraf Tiruan, Kalkulu..."
106,yusi.tyroni.mursityo,"Ir. Yusi Tyroni Mursityo, S.Kom., M.AB.",Departemen Sistem Informasi,Program Studi Sistem Informasi,yusi_tyro@ub.ac.id,[Prediksi Cuaca pada Data Time Series Menggu...,[Pengembangan Aplikasi Mobile Berbasis Android...,[Pelatihan Penggunaan Google Form Untuk Perang...,"[Information System Engineering, Business Proc...",NaN


In [ ]:
from langdetect import detect

# Mendeteksi bahasa dari teks
def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'

# Menghitung jumlah judul berdasarkan bahasa
def count_titles_by_language(df, column_name):
    bahasa_indonesia = 0
    bahasa_inggris = 0

    # Iterasi setiap baris dan deteksi bahasa
    for title in df[column_name]:
        if isinstance(title, list):
            for t in title:
                lang = detect_language(t)
                if lang == 'id':  # Bahasa Indonesia
                    bahasa_indonesia += 1
                elif lang == 'en':  # Bahasa Inggris
                    bahasa_inggris += 1

    return bahasa_indonesia, bahasa_inggris

# Hitung jumlah judul untuk setiap kategori
publikasi_indonesia, publikasi_inggris = count_titles_by_language(profile_df_preprocessed, 'Publikasi')
riset_indonesia, riset_inggris = count_titles_by_language(profile_df_preprocessed, 'Riset')
pengabdian_indonesia, pengabdian_inggris = count_titles_by_language(profile_df_preprocessed, 'Pengabdian')
riwayat_mengajar_indonesia, riwayat_mengajar_inggris = count_titles_by_language(profile_df_preprocessed, 'Riwayat Mengajar')
bidang_keilmuwan_indonesia, bidang_keilmuwan_inggris = count_titles_by_language(profile_df_preprocessed, 'Bidang Keilmuwan')

print(f"Publikasi - Bahasa Indonesia: {publikasi_indonesia}, Bahasa Inggris: {publikasi_inggris}")
print(f"Riset - Bahasa Indonesia: {riset_indonesia}, Bahasa Inggris: {riset_inggris}")
print(f"Pengabdian - Bahasa Indonesia: {pengabdian_indonesia}, Bahasa Inggris: {pengabdian_inggris}")
print(f"Riwayat Mengajar - Bahasa Indonesia: {riwayat_mengajar_indonesia}, Bahasa Inggris: {riwayat_mengajar_inggris}")
print(f"Bidang Keilmuwan - Bahasa Indonesia: {bidang_keilmuwan_indonesia}, Bahasa Inggris: {bidang_keilmuwan_inggris}")

Publikasi - Bahasa Indonesia: 156, Bahasa Inggris: 292
Riset - Bahasa Indonesia: 396, Bahasa Inggris: 38
Pengabdian - Bahasa Indonesia: 554, Bahasa Inggris: 0
Riwayat Mengajar - Bahasa Indonesia: 79, Bahasa Inggris: 4
Bidang Keilmuwan - Bahasa Indonesia: 127, Bahasa Inggris: 129


In [ ]:
def preprocess_text(text):
    if isinstance(text, str):
        # Ubah menjadi huruf kecil
        text = text.lower()

        # Menghapus karakter non-alfabet (angka, tanda baca, dll)
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenisasi teks menjadi kata-kata
        words = text.split()

        # Menghapus stopwords
        stop_words = set(stopwords.words('english') + stopwords.words('indonesian'))
        words = [word for word in words if word not in stop_words]

        # Lematisasi atau stemming tergantung bahasa
        processed_words = []
        for word in words:
            try:
                # Deteksi bahasa teks (Indonesia atau Inggris)
                language = detect(text)

                if language == 'en':  # Jika Bahasa Inggris
                    lemmatized_word = wnl.lemmatize(word)  # Lematisasi Bahasa Inggris
                    processed_words.append(lemmatized_word)
                elif language == 'id':  # Jika Bahasa Indonesia
                    stemmed_word = stemmer_id.stem(word)  # Stemming Bahasa Indonesia
                    processed_words.append(stemmed_word)
            except:
                # Jika deteksi bahasa gagal, lemmatize stem semua kata
                processed_words.append(word)

        return " ".join(processed_words)
    return ''

In [ ]:
# Menggabungkan kolom-kolom yang relevan menjadi satu kolom 'Combined'
profile_df_preprocessed['Combined'] = (
    profile_df_preprocessed['Publikasi'].astype(str) + ' ' +
    profile_df_preprocessed['Riset'].astype(str) + ' ' +
    profile_df_preprocessed['Pengabdian'].astype(str) + ' ' +
    profile_df_preprocessed['Riwayat Mengajar'].astype(str) + ' ' +
    profile_df_preprocessed['Bidang Keilmuwan'].astype(str)
)

profile_df_preprocessed[['Publikasi', 'Riset', 'Pengabdian', 'Riwayat Mengajar', 'Bidang Keilmuwan', 'Combined']].head()

,Publikasi,Riset,Pengabdian,Riwayat Mengajar,Bidang Keilmuwan,Combined
0,[Using Strongly Typed Genetic Programming for ...,[Studi Eksplorasi Hubungan Web Accesibility da...,[Pelatihan Pemodelan Kebutuhan Perangkat Lunak...,NaN,"[Mining Repositori Software, Software Testing]","[""Using Strongly Typed Genetic Programming for..."
1,[The Extended Dijkstra's-based Load Balancing ...,[Mekanisme Penjaminan Kerahasiaan dan Integrit...,[Pelatihan Pengelolaan Layanan Dasar Cloud Com...,NaN,"[Internet of Things, Software-Defined Networki...","[""The Extended Dijkstra's-based Load Balancing..."
2,[Peringkasan Dokumen Bahasa Indonesia Berbasis...,[Evaluasi Detektor-Deskriptor Untuk Kasus Obje...,[Implementasi Sistem Keamanan dengan Closed-Ci...,NaN,"[Sistem Temu Kembali Informasi, Bioinformatika...",['Peringkasan Dokumen Bahasa Indonesia Berbasi...
3,[An Exploratory Study of Requirements Engineer...,[Implementasi Metode Bates Dalam Purwarupa Apl...,[Pelatihan Sistem Pengelolaan Nilai Kurikulum ...,NaN,NaN,['An Exploratory Study of Requirements Enginee...
4,NaN,[Mekanisme Penjaminan Kerahasiaan dan Integrit...,[Pelatihan Pengelolaan Layanan Dasar Cloud Com...,NaN,"[internet of things, cloud computing, cybersec...",nan ['Mekanisme Penjaminan Kerahasiaan dan Int...


In [ ]:
# Memproses data pada profile_df_preprocessed
profile_df_preprocessed['Processed_Combined'] = profile_df_preprocessed['Combined'].apply(preprocess_text)

profile_df_preprocessed[['Combined', 'Processed_Combined']].head()

,Combined,Processed_Combined
0,"[""Using Strongly Typed Genetic Programming for...",using strongly typed genetic programming knowl...
1,"[""The Extended Dijkstra's-based Load Balancing...",extended dijkstrasbased load balancing openflo...
2,['Peringkasan Dokumen Bahasa Indonesia Berbasi...,ringkas dokumen bahasa indonesia bas faktor ma...
3,['An Exploratory Study of Requirements Enginee...,exploratory study requirements engineering pra...
4,nan ['Mekanisme Penjaminan Kerahasiaan dan Int...,nan mekanisme jamin rahasia integritas data io...


# Penerapan Model

## LDA

In [ ]:
# TF-IDF untuk dataset dosen
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(profile_df_preprocessed['Processed_Combined'])

# LDA
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(X)

# Menampilkan topik-topik yang ditemukan oleh LDA
terms = vectorizer.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    print(f"Topik {idx+1}:")
    print([terms[i] for i in topic.argsort()[::-1]])

Topik 1:
['sensor', 'things', 'internet', 'data', 'iot', 'sistem', 'bangun', 'tengah', 'protokol', 'kota', 'juru', 'malang', 'node', 'komunikasi', 'wireless', 'kirim', 'jaring', 'bas', 'sekolah', 'latih', 'computing', 'komputer', 'kembang', 'rancang', 'cloud', 'mqtt', 'nan', 'simpan', 'ble', 'low', 'bluetooth', 'upnp', 'pi', 'raspberry', 'kriptografi', 'anwar', 'reservasi', 'dr', 'rujuk', 'saiful', 'jantung', 'streaming', 'mekanisme', 'distribusi', 'routing', 'esp', 'energy', 'arsitektur', 'infrastruktur', 'networking', 'mesh', 'pasien', 'moodle', 'instalasi', 'elearning', 'aman', 'moodlebox', 'konservasi', 'mangrove', 'selatan', 'dinas', 'nirkabel', 'mongodb', 'cassandra', 'heterogen', 'hbase', 'media', 'kendara', 'padu', 'komunitas', 'bagi', 'aws', 'amazon', 'gedung', 'pencil', 'rumah', 'ambulance', 'modul', 'audit', 'perangkat', 'kelola', 'sinyal', 'elektronik', 'cryptography', 'lightweight', 'service', 'area', 'purwarupa', 'sakit', 'banding', 'siswa', 'buffer', 'ehealth', 'prehospi

## Cosine Similarity

In [ ]:
def rekomendasi_dosen(input_mahasiswa, vectorizer, profile_df_preprocessed, X, top_n=5):
    # TF-IDF untuk input mahasiswa
    X_mahasiswa = vectorizer.transform([input_mahasiswa])

    # Menghitung similarity antara input mahasiswa dan dosen
    similarity_scores = cosine_similarity(X_mahasiswa, X)

    # Menentukan 5 dosen teratas berdasarkan similarity
    recommended_dosen_indices = similarity_scores[0].argsort()[-top_n:][::-1]

    recommended_dosen = []
    for idx in recommended_dosen_indices:
        recommended_dosen.append({
            'Nama Dosen': profile_df_preprocessed.iloc[idx]['Nama'],
            'Email Dosen': profile_df_preprocessed.iloc[idx].get('Email', 'Tidak tersedia'),
            'Bidang Keilmuwan': profile_df_preprocessed.iloc[idx].get('Bidang Keilmuwan', 'Tidak tersedia'),
            'Skor Similarity': similarity_scores[0][idx]
        })

    recommended_df = pd.DataFrame(recommended_dosen)
    return recommended_df

## Evaluasi

In [ ]:
def get_ground_truth_from_recommendations(recommended_df):
    # Menampilkan daftar dosen yang direkomendasikan
    nama_dosen = recommended_df['Nama Dosen'].tolist()
    print("Daftar dosen yang direkomendasikan:")
    for i, nama in enumerate(nama_dosen, 1):
        print(f"{i}. {nama}")

    # Meminta inputan nomor urut berdasarkan ranking
    ranking_input = input("Masukkan nomor dosen berdasarkan ranking (pisahkan dengan koma): ")
    ranking_indices = [int(num.strip()) - 1 for num in ranking_input.split(",")]

    # Membuat ranking ground truth dari inputan
    ground_truth_ranking = [nama_dosen[idx] for idx in ranking_indices]

    return ground_truth_ranking

In [ ]:
from scipy.stats import spearmanr

def evaluate_recommendation(input_mahasiswa, vectorizer, profile_df_preprocessed, X, top_n=5):
    # Mendapatkan rekomendasi dosen
    recommended_df = rekomendasi_dosen(input_mahasiswa, vectorizer, profile_df_preprocessed, X, top_n)

    # Mendapatkan ranking ground truth dari inputan
    ground_truth_ranking = get_ground_truth_from_recommendations(recommended_df)

    # Mengambil nama dosen dari rekomendasi model
    model_ranking = recommended_df['Nama Dosen'].tolist()

    # Membuat urutan peringkat untuk model dan ground truth berdasarkan posisi
    model_ranking_indices = {name: idx + 1 for idx, name in enumerate(model_ranking)}  # Peringkat model
    ground_truth_ranking_indices = {name: idx + 1 for idx, name in enumerate(ground_truth_ranking)}  # Peringkat ground truth

    # Mengonversi nama dosen ke peringkat numerik yang sesuai
    model_ranking_numeric = [model_ranking_indices.get(name, -1) for name in ground_truth_ranking]
    ground_truth_ranking_numeric = [ground_truth_ranking_indices.get(name, -1) for name in ground_truth_ranking]

    # Menghitung Spearman Rank Correlation antara ranking model dan ground truth
    correlation, _ = spearmanr(model_ranking_numeric, ground_truth_ranking_numeric)

    # Kembalikan hasil evaluasi
    return correlation, recommended_df

In [ ]:
# Contoh inputan mahasiswa (judul dan abstrak)
judul_mahasiswa_1 = "Pengembangan Aplikasi Sistem Informasi Akademik Mobile dengan Pendekatan User-Centered Design"
abstrak_mahasiswa_1 = "Kebutuhan akan sistem informasi akademik yang mudah diakses melalui perangkat mobile semakin meningkat. Penelitian ini mengembangkan aplikasi sistem informasi akademik berbasis Android dengan pendekatan User-Centered Design (UCD). Tahapan penelitian meliputi analisis kebutuhan, desain prototipe, implementasi, dan evaluasi. Hasil uji coba menunjukkan bahwa aplikasi ini memiliki tingkat kepuasan pengguna sebesar 92%, terutama dalam fitur pencarian jadwal kuliah, pengelolaan nilai, dan pengumuman kampus."

# Preprocessing inputan mahasiswa
input_mahasiswa_1 = preprocess_text(judul_mahasiswa_1 + " " + abstrak_mahasiswa_1)

In [ ]:
recommended_df_1 = rekomendasi_dosen(input_mahasiswa_1, vectorizer, profile_df_preprocessed, X)
recommended_df_1

,Nama Dosen,Email Dosen,Bidang Keilmuwan,Skor Similarity
0,"Mochamad Chandra Saputra, S.Kom., M.Eng., M.T....",andra@ub.ac.id,"[Pengujian Perangkat Lunak, Rekayasa Perangkat...",0.222621
1,"Agi Putra Kharisma, S.T., M.T.",agi@ub.ac.id,"[Human-Computer Interaction, Ergonomics/Human ...",0.212611
2,"Ir. Lutfi Fanani, S.Kom., M.T., M.Sc.",lutfifanani@ub.ac.id,"[UI/UX Design and Evaluation, Human-Computer I...",0.201132
3,"Ir. Yusi Tyroni Mursityo, S.Kom., M.AB.",yusi_tyro@ub.ac.id,"[Information System Engineering, Business Proc...",0.197415
4,"Ir. Komang Candra Brata, S.Kom., M.T., M.Sc.",k.candra.brata@ub.ac.id,"[AR/VR, UI/UX, HCI, Software Engineering, Mobi...",0.187825


In [ ]:
# Menjalankan evaluasi rekomendasi
correlation, recommended_df = evaluate_recommendation(
    input_mahasiswa_1, vectorizer, profile_df_preprocessed, X, top_n=5
)

print(f"Spearman Rank Correlation: {correlation:.4f}")

Daftar dosen yang direkomendasikan:
1. Mochamad Chandra Saputra, S.Kom., M.Eng., M.T., Ph.D
2. Agi Putra Kharisma, S.T., M.T.
3. Ir. Lutfi Fanani, S.Kom., M.T., M.Sc.
4. Ir. Yusi Tyroni Mursityo, S.Kom., M.AB.
5. Ir. Komang Candra Brata, S.Kom., M.T., M.Sc.
Masukkan nomor dosen berdasarkan ranking (pisahkan dengan koma): 2,1,3,4,5
Spearman Rank Correlation: 0.9000


In [ ]:
# Contoh inputan mahasiswa (judul dan abstrak)
judul_mahasiswa_2 = "Implementasi Algoritma Deep Learning untuk Deteksi Emosi dari Teks Menggunakan LSTM"
abstrak_mahasiswa_2 = "Deteksi emosi dari teks merupakan salah satu aplikasi penting dalam bidang pengolahan bahasa alami (Natural Language Processing) yang dapat digunakan untuk analisis sentimen, chatbot, dan sistem rekomendasi. Penelitian ini bertujuan untuk mengimplementasikan algoritma Long Short-Term Memory (LSTM), salah satu jenis jaringan saraf tiruan, untuk mendeteksi emosi dalam teks. Dataset yang digunakan adalah kumpulan teks berbahasa Indonesia dengan label emosi seperti bahagia, sedih, marah, dan takut. Proses penelitian meliputi prapemrosesan data, seperti tokenisasi dan stopword removal, pelatihan model menggunakan LSTM, serta evaluasi performa model dengan metrik akurasi, presisi, recall, dan F1-score. Hasil eksperimen menunjukkan bahwa model LSTM mampu mencapai akurasi sebesar 85% pada data uji, membuktikan efektivitasnya dalam mengenali pola emosi dari teks berbahasa Indonesia. Penelitian ini diharapkan dapat menjadi dasar pengembangan aplikasi berbasis AI yang mampu memahami emosi pengguna secara lebih mendalam."

# Preprocessing inputan mahasiswa
input_mahasiswa_2 = preprocess_text(judul_mahasiswa_2 + " " + abstrak_mahasiswa_2)

In [ ]:
recommended_df_2 = rekomendasi_dosen(input_mahasiswa_2, vectorizer, profile_df_preprocessed, X)
recommended_df_2

,Nama Dosen,Email Dosen,Bidang Keilmuwan,Skor Similarity
0,"Tirana Noor Fatyanosa, S.Kom., M.Kom., Ph.D.",fatyanosa@ub.ac.id,"[Deep Neuroevolution, Meta-learning, Evolution...",0.271620
1,"Dr. Drs. Ir. Achmad Ridok, M.Kom.",acridokb@ub.ac.id,"[Sistem Temu Kembali Informasi, Bioinformatika...",0.248636
2,"Putra Pandu Adikara, S.Kom., M.Kom.",adikara.putra@ub.ac.id,[Pemerolehan Informasi (Information Retrieval)...,0.168350
3,"Nurul Hidayat, S.Pd., M.Sc.",ntayadih@ub.ac.id,"[Sistem Pakar, analisis sentimen, analisis kes...",0.133120
4,"Ir. Zainul Abidin, S.T., M.Eng., M.T., Ph.D.",zainulabidin@ub.ac.id,"[pemrosesan sinyal biomedik, sistem tertanam, ...",0.130112


In [ ]:
# Menjalankan evaluasi rekomendasi
correlation, recommended_df = evaluate_recommendation(
    input_mahasiswa_2, vectorizer, profile_df_preprocessed, X, top_n=5
)

print(f"Spearman Rank Correlation: {correlation:.4f}")

Daftar dosen yang direkomendasikan:
1. Tirana Noor Fatyanosa, S.Kom., M.Kom., Ph.D.
2. Dr. Drs. Ir. Achmad Ridok, M.Kom.
3. Putra Pandu Adikara, S.Kom., M.Kom.
4. Nurul Hidayat, S.Pd., M.Sc.
5. Ir. Zainul Abidin, S.T., M.Eng., M.T., Ph.D.
Masukkan nomor dosen berdasarkan ranking (pisahkan dengan koma): 1,2,3,4,5
Spearman Rank Correlation: 1.0000


In [ ]:
# Contoh inputan mahasiswa (judul dan abstrak)
judul_mahasiswa_3 = "Penerapan Teknologi IoT untuk Pemantauan Kesehatan Tanaman pada Sistem Pertanian Cerdas"
abstrak_mahasiswa_3 = "Kemajuan teknologi Internet of Things (IoT) telah memberikan dampak signifikan pada berbagai bidang, termasuk pertanian. Penelitian ini bertujuan untuk merancang dan mengimplementasikan sistem IoT untuk memantau kesehatan tanaman secara real-time. Sistem ini menggunakan sensor untuk mengukur parameter lingkungan seperti kelembaban tanah, suhu, dan intensitas cahaya, yang diintegrasikan dengan mikrokontroler berbasis NodeMCU. Data yang diperoleh dikirim ke platform cloud untuk dianalisis dan ditampilkan melalui aplikasi berbasis web. Pengujian dilakukan pada tanaman tomat di lingkungan terkendali selama 30 hari, dan hasil menunjukkan bahwa sistem ini dapat memberikan informasi akurat untuk membantu petani mengambil keputusan yang tepat. Sistem ini diharapkan menjadi solusi untuk meningkatkan produktivitas dan efisiensi dalam pertanian modern."

# Preprocessing inputan mahasiswa
input_mahasiswa_3 = preprocess_text(judul_mahasiswa_3 + " " + abstrak_mahasiswa_3)

In [ ]:
recommended_df_3 = rekomendasi_dosen(input_mahasiswa_3, vectorizer, profile_df_preprocessed, X)
recommended_df_3

,Nama Dosen,Email Dosen,Bidang Keilmuwan,Skor Similarity
0,"Eko Sakti Pramukantoro, S.Kom., M.Kom., Ph.D.",ekosakti@ub.ac.id,NaN,0.234135
1,"Kasyful Amron, S.T., M.Sc.",kasyful@ub.ac.id,"[Wireless Networks, Wireless Sensor Networks, ...",0.233570
2,"Reza Andria Siregar, S.T., M.Kom.",reza.jalin@ub.ac.id,"[Jaringan Komputer, Jaringan Bergerak, Jaringa...",0.232385
3,"Ir. Komang Candra Brata, S.Kom., M.T., M.Sc.",k.candra.brata@ub.ac.id,"[AR/VR, UI/UX, HCI, Software Engineering, Mobi...",0.230371
4,"Ratih Kartika Dewi, S.T., M.Kom.",ratihkartikad@ub.ac.id,[Artificial Intelligence in Multimedia],0.219414


In [ ]:
# Menjalankan evaluasi rekomendasi
correlation, recommended_df = evaluate_recommendation(
    input_mahasiswa_3, vectorizer, profile_df_preprocessed, X, top_n=5
)

print(f"Spearman Rank Correlation: {correlation:.4f}")

Daftar dosen yang direkomendasikan:
1. Eko Sakti Pramukantoro, S.Kom., M.Kom., Ph.D.
2. Kasyful Amron, S.T., M.Sc.
3. Reza Andria Siregar, S.T., M.Kom.
4. Ir. Komang Candra Brata, S.Kom., M.T., M.Sc.
5. Ratih Kartika Dewi, S.T., M.Kom.
Masukkan nomor dosen berdasarkan ranking (pisahkan dengan koma): 1,2,3,4,5
Spearman Rank Correlation: 1.0000


In [ ]:
# Contoh inputan mahasiswa (judul dan abstrak)
judul_mahasiswa_4 = "Pengembangan Aplikasi Game Edukasi Interaktif Menggunakan Unity untuk Meningkatkan Pemahaman Matematika Dasar pada Anak"
abstrak_mahasiswa_4 = "Penggunaan game edukasi dalam pembelajaran anak-anak semakin populer karena dapat membuat proses belajar menjadi lebih menarik dan menyenangkan. Penelitian ini bertujuan untuk mengembangkan aplikasi game edukasi interaktif menggunakan Unity yang bertujuan untuk meningkatkan pemahaman matematika dasar pada anak usia 6 hingga 10 tahun. Game ini dirancang dengan berbagai level yang mengajarkan konsep-konsep matematika dasar, seperti penjumlahan, pengurangan, perkalian, dan pembagian, dengan menggunakan elemen visual dan audio yang menarik. Pengujian dilakukan dengan melibatkan anak-anak sebagai pengguna untuk mengukur dampaknya terhadap peningkatan pemahaman matematika dasar. Hasil penelitian menunjukkan bahwa aplikasi game ini dapat meningkatkan minat belajar dan pemahaman anak terhadap materi matematika dasar secara signifikan."

# Preprocessing inputan mahasiswa
input_mahasiswa_4 = preprocess_text(judul_mahasiswa_4 + " " + abstrak_mahasiswa_4)

In [ ]:
recommended_df_4 = rekomendasi_dosen(input_mahasiswa_4, vectorizer, profile_df_preprocessed, X)
recommended_df_4

,Nama Dosen,Email Dosen,Bidang Keilmuwan,Skor Similarity
0,"Wibisono Sukmo Wardhono, S.T., M.T.",wibiwardhono@ub.ac.id,"[Augmented Reality, Virtual Reality, Pengemban...",0.228557
1,"Muhammad Aminul Akbar, S.Kom., M.T",muhammad.aminul@ub.ac.id,"[Artificial Intelligence for Game, Interaksi M...",0.169322
2,"Eriq Muhammad Adams Jonemaro, S.T., M.Kom.",eriq.adams@ub.ac.id,"[Serious Game, Extended Reality, dan Human Com...",0.169237
3,"Aswin Suharsono, S.T., M.T.",aswin@ub.ac.id,[Machine Learning],0.152528
4,"Ir. Tri Afirianto, S.T., M.T.",tri.afirianto@ub.ac.id,"[Realitas Berimbuh, Realitas Maya, Gim Edukasi]",0.142747


In [ ]:
# Menjalankan evaluasi rekomendasi
correlation, recommended_df = evaluate_recommendation(
    input_mahasiswa_4, vectorizer, profile_df_preprocessed, X, top_n=5
)

print(f"Spearman Rank Correlation: {correlation:.4f}")

Daftar dosen yang direkomendasikan:
1. Wibisono Sukmo Wardhono, S.T., M.T.
2. Muhammad Aminul Akbar, S.Kom., M.T
3. Eriq Muhammad Adams Jonemaro, S.T., M.Kom.
4. Aswin Suharsono, S.T., M.T.
5. Ir. Tri Afirianto, S.T., M.T.
Masukkan nomor dosen berdasarkan ranking (pisahkan dengan koma): 3,1,2,4,5
Spearman Rank Correlation: 0.7000


In [ ]:
# Contoh inputan mahasiswa (judul dan abstrak)
judul_mahasiswa_5 = "Analisis dan Perancangan Sistem Pemesanan Makanan Online Menggunakan Use Case Diagram dan UML"
abstrak_mahasiswa_5 = "Perkembangan teknologi internet mendorong munculnya berbagai layanan pemesanan makanan secara online. Penelitian ini bertujuan untuk merancang sistem pemesanan makanan online menggunakan Use Case Diagram dan Unified Modeling Language (UML) untuk menggambarkan proses interaksi antara pengguna, sistem, dan penyedia layanan. Fitur utama sistem ini meliputi pemilihan menu, proses pemesanan, pembayaran, serta pelacakan status pesanan. Dengan menggunakan Use Case Diagram, penelitian ini mengidentifikasi aktor-aktor yang terlibat, seperti pelanggan, admin, dan kurir, serta menggambarkan skenario-skenario yang mungkin terjadi dalam sistem. Hasil penelitian ini diharapkan dapat memberikan solusi yang efisien untuk mempercepat proses pemesanan dan pengiriman makanan, serta meningkatkan kepuasan pelanggan."

# Preprocessing inputan mahasiswa
input_mahasiswa_5 = preprocess_text(judul_mahasiswa_5 + " " + abstrak_mahasiswa_5)

In [ ]:
recommended_df_5 = rekomendasi_dosen(input_mahasiswa_5, vectorizer, profile_df_preprocessed, X)
recommended_df_5

,Nama Dosen,Email Dosen,Bidang Keilmuwan,Skor Similarity
0,"Bayu Priyambadha, S.Kom., M.Kom., Ph.D.",bayu_priyambadha@ub.ac.id,"[Software Design, Software Maintenance, Softwa...",0.173125
1,"Dr. Himawat Aryadita, S.T., M.Sc.",himawat@ub.ac.id,"[E-Business, Manajemen Produk, Manajemen dan O...",0.123342
2,"Tirana Noor Fatyanosa, S.Kom., M.Kom., Ph.D.",fatyanosa@ub.ac.id,"[Deep Neuroevolution, Meta-learning, Evolution...",0.110617
3,"Dr.Eng. Novanto Yudistira, S.Kom., M.Sc.",yudistira@ub.ac.id,"[komputer visi, deep learning, big data]",0.102763
4,"Ir. Tri Astoto Kurniawan, S.T., M.T., Ph.D.",triak@ub.ac.id,"[Rekayasa Kebutuhan, Software Reuse, Service C...",0.100431


In [ ]:
# Menjalankan evaluasi rekomendasi
correlation, recommended_df = evaluate_recommendation(
    input_mahasiswa_5, vectorizer, profile_df_preprocessed, X, top_n=5
)

print(f"Spearman Rank Correlation: {correlation:.4f}")

Daftar dosen yang direkomendasikan:
1. Bayu Priyambadha, S.Kom., M.Kom., Ph.D.
2. Dr. Himawat Aryadita, S.T., M.Sc.
3. Tirana Noor Fatyanosa, S.Kom., M.Kom., Ph.D.
4. Dr.Eng. Novanto Yudistira, S.Kom., M.Sc.
5. Ir. Tri Astoto Kurniawan, S.T., M.T., Ph.D.
Masukkan nomor dosen berdasarkan ranking (pisahkan dengan koma): 1, 2, 5, 4, 3
Spearman Rank Correlation: 0.6000


In [ ]:
# Korelasi dari setiap eksperimen
correlation_1 = 0.9000  # Dari eksperimen pertama
correlation_2 = 1.0000  # Dari eksperimen kedua
correlation_3 = 1.0000  # Dari eksperimen ketiga
correlation_4 = 0.7000  # Dari eksperimen keempat
correlation_5 = 0.6000  # Dari eksperimen kelima

# Hitung rata-rata Spearman Rank Correlation
average_correlation = (correlation_1 + correlation_2 + correlation_3 + correlation_4 + correlation_5) / 5

# Output untuk menampilkan hasil evaluasi
print("Hasil Evaluasi Kinerja Model Sistem Rekomendasi Dosen:")
print(f"Spearman Rank Correlation untuk Eksperimen 1: {correlation_1:.4f}")
print(f"Spearman Rank Correlation untuk Eksperimen 2: {correlation_2:.4f}")
print(f"Spearman Rank Correlation untuk Eksperimen 3: {correlation_3:.4f}")
print(f"Spearman Rank Correlation untuk Eksperimen 4: {correlation_4:.4f}")
print(f"Spearman Rank Correlation untuk Eksperimen 5: {correlation_5:.4f}")
print(f"Rata-rata Spearman Rank Correlation: {average_correlation:.4f}")

# Menyimpulkan kinerja model secara keseluruhan
if average_correlation >= 0.90:
    print("\nKesimpulan: Model sistem rekomendasi dosen ini menunjukkan kinerja yang sangat baik dengan rata-rata korelasi Spearman di atas 0.90.")
elif 0.75 <= average_correlation < 0.90:
    print("\nKesimpulan: Model sistem rekomendasi dosen ini menunjukkan kinerja yang baik dengan rata-rata korelasi Spearman antara 0.75 hingga 0.90.")
else:
    print("\nKesimpulan: Model sistem rekomendasi dosen ini memerlukan perbaikan, dengan rata-rata korelasi Spearman di bawah 0.75.")

Hasil Evaluasi Kinerja Model Sistem Rekomendasi Dosen:
Spearman Rank Correlation untuk Eksperimen 1: 0.9000
Spearman Rank Correlation untuk Eksperimen 2: 1.0000
Spearman Rank Correlation untuk Eksperimen 3: 1.0000
Spearman Rank Correlation untuk Eksperimen 4: 0.7000
Spearman Rank Correlation untuk Eksperimen 5: 0.6000
Rata-rata Spearman Rank Correlation: 0.8400

Kesimpulan: Model sistem rekomendasi dosen ini menunjukkan kinerja yang baik dengan rata-rata korelasi Spearman antara 0.75 hingga 0.90.
